# ZeroxPDFLoader

This notebook provides a quick overview for getting started with `ZeroxPDF` [document loader](https://python.langchain.com/docs/concepts/document_loaders). For detailed documentation of all DocumentLoader features and configurations head to the [API reference](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.ZeroxPDFLoader.html).

  

## Overview
`ZeroxPDFLoader` is a document loader that leverages the [Zerox](https://github.com/getomni-ai/zerox) library. Zerox converts PDF documents into images, processes them using a vision-capable language model, and generates a structured Markdown representation. This loader allows for asynchronous operations and provides page-level document extraction.

### Integration details

| Class                                                                                                                                             | Package | Local | Serializable | JS support|
|:--------------------------------------------------------------------------------------------------------------------------------------------------| :--- | :---: | :---: |  :---: |
| [PyPDFLoader](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.ZeroxPDFLoader.html) | [langchain_community](https://python.langchain.com/api_reference/community/index.html) | ❌ | ❌ | ❌ |   
   
---------   

### Loader features

|   Source    | Document Lazy Loading | Native Async Support | Extract Images | Extract Tables |
|:-----------:| :---: | :---: | :---: |:---: |
| PyPDFLoader | ✅ | ❌ | ✅ | ✅  |

  

## Setup

### Credentials

Appropriate credentials need to be set up in environment variables. The loader supports number of different models and model providers. See _Usage_ header below to see few examples or [Zerox documentation](https://github.com/getomni-ai/zerox) for a full list of supported models.

### Installation

To use `ZeroxPDFLoader`, you need to install the `zerox` package. Also make sure to have `langchain-community` and `py-zerox`installed.

In [ ]:
%pip install -qU langchain_community py-zerox
%pip install -qq ../../../../dist/patch_langchain_pdf_loader*.whl

## Initialization

`ZeroxPDFLoader` enables PDF text extraction using vision-capable language models by converting each page into an image and processing it asynchronously. To use this loader, you need to specify a model and configure any necessary environment variables for Zerox, such as API keys.

If you're working in an environment like Jupyter Notebook, you may need to handle asynchronous code by using `nest_asyncio`. You can set this up as follows:

```python
import nest_asyncio
nest_asyncio.apply()
```

In [2]:
import os
from getpass import getpass
import nest_asyncio
from patch_langchain_community.document_loaders import ZeroxPDFLoader

nest_asyncio.apply()

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your API key: ")
file_path = "./example_data/layout-parser-paper.pdf"
loader = ZeroxPDFLoader(file_path)

## Load

In [3]:
docs = loader.load()
docs[0]

images_to_text can not be simulate



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



ModelAccessError: 
    Your provided model can't be accessed. Please make sure you have access to the model and also required environment variables are setup correctly including valid api key(s).
    Refer: https://docs.litellm.ai/docs/providers
     (Extra Info: {'model': 'gpt-4o-mini'})

In [ ]:
import pprint

pprint.pp(docs[0].metadata)

## Lazy Load


In [ ]:
pages = []
for doc in loader.lazy_load():
    pages.append(doc)
    if len(pages) >= 10:
        # do some paged operation, e.g.
        # index.upsert(page)

        pages = []
len(pages)

*Metadata attribute now contains several pieces of information about the file in addition to the total number of pages.*   
*Why is this important? If we want to reference a document, we need to determine if it’s relevant. A reference is valid if it helps the user quickly locate the fragment within the document (using the page and/or a chunk excerpt). But if the URL points to a PDF file without a page number (for various reasons) and the file has a large number of pages, we want to remove the reference that doesn’t assist the user. There’s no point in referencing a 100-page document! The total_pages metadata can then be used.*

In [ ]:
pprint.pp(pages[0].page_content[:100])
pprint.pp(pages[0].metadata)

## Extract the PDF by page. Each page is extracted as a langchain Document object:

In [ ]:
loader = ZeroxPDFLoader(
    "./example_data/layout-parser-paper.pdf",
    mode="page",
)
docs = loader.load()
print(len(docs))

## Extract the whole PDF as a single langchain Document object:

In [ ]:
loader = ZeroxPDFLoader(
    "./example_data/layout-parser-paper.pdf",
    mode="single",
)
docs = loader.load()
print(len(docs))

## Add a custom *pages_delimitor* to identify where are ends of pages in *single* mode:

In [ ]:
loader = ZeroxPDFLoader(
    "./example_data/layout-parser-paper.pdf",
    mode="single",
    pages_delimitor="\n-------THIS IS A CUSTOM END OF PAGE-------\n",
)
docs = loader.load()
print(docs[0].page_content[:5780])

*This could simply be \n, or \f to clearly indicate a page change, or \<!-- PAGE BREAK --> for seamless injection in a Markdown viewer without a visual effect.*

*Why is it important to identify page breaks when retrieving the full document flow? Because we generally want to provide a URL with the chunk’s location when the LLM answers. While it’s possible to reference the entire PDF, this isn’t practical if it’s more than two pages long. It’s better to indicate the specific page to display in the URL. Therefore, assistance is needed so that chunking algorithms can add the page metadata to each chunk. The choice of delimiter helps the algorithm prioritize this parameter.*

# Extract images from the PDF

*In LangChain the OCR process used by the parsers involves asking the parser for the text on a page, then retrieving images to apply OCR.   
In the previous implementation the text extracted from images was appended to the end of the page text. In a RAG context it means if in the original document a paragraph is spread across two pages it would have been cut in half by the OCR process putting the text (from the image) in between, worsening RAG model's performance.*    
   
*To avoid this, we modified the strategy for injecting OCR results from images. Now, the result is inserted between the last and the second-to-last paragraphs of text (\n\n or \n) of the page.*

## Extract images from the PDF in *markdown* format (can also be *html* or *text*) with vision model: 

In [ ]:
from patch_langchain_community.document_loaders.parsers.pdf import (
    convert_images_to_description,
)

loader = ZeroxPDFLoader(
    "./example_data/layout-parser-paper.pdf",
    mode="page",
    extract_images=True,
    images_to_text=convert_images_to_description(model=None, format="html"),
)
docs = loader.load()

print(docs[5].page_content)

## API reference

### `ZeroxPDFLoader`

This loader class initializes with a file path and model type, and supports custom configurations via `zerox_kwargs` for handling Zerox-specific parameters.

**Arguments**:
- `file_path` (Union[str, Path]): Path to the PDF file.
- `model` (str): Vision-capable model to use for processing in format `<provider>/<model>`.
Some examples of valid values are: 
  - `model = "gpt-4o-mini" ## openai model`
  - `model = "azure/gpt-4o-mini"`
  - `model = "gemini/gpt-4o-mini"`
  - `model="claude-3-opus-20240229"`
  - `model = "vertex_ai/gemini-1.5-flash-001"`
  - See more details in [Zerox documentation](https://github.com/getomni-ai/zerox)
  - Defaults to `"gpt-4o-mini".`
- `**zerox_kwargs` (dict): Additional Zerox-specific parameters such as API key, endpoint, etc.
  - See [Zerox documentation](https://github.com/getomni-ai/zerox)

**Methods**:
- `lazy_load`: Generates an iterator of `Document` instances, each representing a page of the PDF, along with metadata including page number and source.

See full API documentaton [here](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.ZeroxPDFLoader.html)

## Notes
- **Model Compatibility**: Zerox supports a range of vision-capable models. Refer to [Zerox's GitHub documentation](https://github.com/getomni-ai/zerox) for a list of supported models and configuration details.
- **Environment Variables**: Make sure to set required environment variables, such as `API_KEY` or endpoint details, as specified in the Zerox documentation.
- **Asynchronous Processing**: If you encounter errors related to event loops in Jupyter Notebooks, you may need to apply `nest_asyncio` as shown in the setup section.


## Troubleshooting
- **RuntimeError: This event loop is already running**: Use `nest_asyncio.apply()` to prevent asynchronous loop conflicts in environments like Jupyter.
- **Configuration Errors**: Verify that the `zerox_kwargs` match the expected arguments for your chosen model and that all necessary environment variables are set.

## Additional Resources
- **Zerox Documentation**: [Zerox GitHub Repository](https://github.com/getomni-ai/zerox)
- **LangChain Document Loaders**: [LangChain Documentation](https://python.langchain.com/docs/integrations/document_loaders/)